In [1]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
import ctypes
from pycuda.compiler import SourceModule
import time
#define CLIP(x, min, max) (fminf(fmaxf(x, min), max))

N = 50  # Increase to capture more user preferences
  # Adjust as needed
totalMoviesUsingPerPerson=400

# Define the Row structure
class Row(ctypes.Structure):
    _fields_ = [("col1", ctypes.c_int), ("col2", ctypes.c_float)]
class RowTest(ctypes.Structure):
    _fields_ = [("col1", ctypes.c_int), ("col2", ctypes.c_float),("user",ctypes.c_int)]



# Create some example row data
rows_data = []

# Define the parallel matrix multiplication kernel
kernel_code = """
struct Row {
    int col1;
    float col2;
};

__global__ void matmul(float *A, float *B, int N, int raters, int movies, Row *rows, float learning_rate) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;  // Movies
    int col = blockIdx.x * blockDim.x + threadIdx.x;  // Raters
    int totalMoviesUsingPerPerson=400;
    if (row < movies && col < raters && col > 0 && row > 0) {
        int found = 0;
        float actual_rating = 0.0;

        // Find actual rating from stored data
        
        int start = 0;
        int end = totalMoviesUsingPerPerson-1;  // Fix here to prevent out-of-bounds access
        while (start <= end) {
            int mid = start + (end - start) / 2;
            if (rows[(col - 1) * totalMoviesUsingPerPerson + mid].col1 == row){
                found = 1;
                actual_rating = rows[(col - 1) * totalMoviesUsingPerPerson + mid].col2;
                break;
            } else if (rows[(col - 1) * totalMoviesUsingPerPerson + mid].col1==0 || rows[(col - 1) * totalMoviesUsingPerPerson + mid].col1 > row) {
                end = mid - 1;
            } else {
                start = mid + 1;
            }
        }

        if (found) {
            float predicted_rating = 0.0;
            for (int k = 0; k < N; k++) {
                predicted_rating += A[row * N + k] * B[k * raters + col];
            }
            
            float error = actual_rating - predicted_rating;
            
            for (int k = 0; k < N; k++) {
                float lambda = 0.01;
                float regularization = lambda * (A[row * N + k] * A[row * N + k] + B[k * raters + col] * B[k * raters + col]);
                float grad_A = -2.0 * error * B[k * raters + col] + regularization;
                float grad_B = -2.0 * error * A[row * N + k] + regularization;
                                
                // Apply updates
                atomicAdd(&A[row * N + k], -learning_rate * grad_A);
                atomicAdd(&B[k * raters + col], -learning_rate * grad_B);

                atomicExch(&A[row * N + k], fminf(fmaxf(A[row * N + k], -1.0f), 1.0f));
                atomicExch(&B[k * raters + col], fminf(fmaxf(B[k * raters + col], -1.0f), 1.0f));
            }

            

        }

    }
}

"""

mod = SourceModule(kernel_code)


def read_csv_to_list_rating(file_path):
    """Read a CSV file into a list of lists with specific column types."""
    with open(file_path, 'r', encoding='utf-8') as file:
        matrix = []
        header = file.readline().strip().split(',')
        for index,line in enumerate(file):
            values = line.strip().split(',')
            
            row = [int(values[0]), int(values[1]), float(values[2]), int(values[3])]
            matrix.append(row)
        return header, matrix

def read_csv_to_list_movies(file_path):
    """Read a CSV file into a list of lists with specific column types."""
    with open(file_path, 'r', encoding='utf-8') as file:
        matrix = []
        header = file.readline().strip().split(',')
        index=1
        matrix.append([0,"",""])
        for line in (file):
            values = line.strip().split(',')
            while index<int(values[0]):
                matrix.append([0,"",""])
                index+=1
                
            row = [int(values[0]), str(values[1]), str(values[-1])]
            matrix.append(row)
            index+=1
        return header, matrix
headerMovies, matrixMovies = read_csv_to_list_movies('movies.csv')
headerRating, matrixrating = read_csv_to_list_rating('ratings.csv')


totalRaters = int(matrixrating[-1][0])+1  # total rows
totalMovies = int(matrixMovies[-1][0])+1
print(totalRaters)
print(totalMovies)


d = {}
# row 0 is userID

########################################################################## I CHANGED HERE#####################
#
for row in matrixrating:
    if row[0] not in d:
        d[row[0]] = []
        d[row[0]].append(list(row[1:4]))
    else:
        d[row[0]].append(list(row[1:4]))

for key, val in d.items():
    d[key] = sorted(val, key=lambda x: x[2])

for key, val in d.items():
    d[key] = d[key][::-1]

newD = {}
for key, val in d.items():
    newD[key] = sorted(d[key][:totalMoviesUsingPerPerson], key=lambda x: x[0])

totalUsers = len(newD)
total_rat_us=0
rows_data = []
for i in range(1, (len(newD) + 1)):
    row = newD[i]
    for j in range(totalMoviesUsingPerPerson):
        if j < len(row):
            total_rat_us+=1
            rows_data.append(Row(row[j][0], row[j][1]))
        else:
            rows_data.append(Row(0, 0.0))
print("Total rating used ",end="")
print(total_rat_us)
matrixAMovies = np.zeros((totalMovies, N), dtype=np.float32)
matrixBUsers = np.zeros((N, totalRaters), dtype=np.float32)


# Fill matrices with random values in the range -0.01 to 0.01
matrixAMovies = np.random.uniform(-0.1, 0.1, size=(totalMovies, N)).astype(np.float32)
matrixBUsers = np.random.uniform(-0.1, 0.1, size=(N, totalRaters)).astype(np.float32)

matrixAMovies = np.ascontiguousarray(matrixAMovies, dtype=np.float32)
matrixBUsers = np.ascontiguousarray(matrixBUsers, dtype=np.float32)

A_gpu_matrixAMovies = cuda.mem_alloc(matrixAMovies.nbytes)
B_gpu_matrixBUsers = cuda.mem_alloc(matrixBUsers.nbytes)

if A_gpu_matrixAMovies is None or B_gpu_matrixBUsers is None:
    print("Damn")

rows = (Row * len(rows_data))(*rows_data)

# Allocate memory on the GPU for the rows
rows_gpu = cuda.mem_alloc(ctypes.sizeof(rows))
if rows_gpu is None:
    print("damn")

# Transfer the rows data to the GPU
cuda.memcpy_htod(rows_gpu, rows)

cuda.memcpy_htod(A_gpu_matrixAMovies, matrixAMovies)
cuda.memcpy_htod(B_gpu_matrixBUsers, matrixBUsers)


matmul = mod.get_function("matmul")

# Define grid and block sizes
block_size = (16, 16, 1)  # 16x16 threads per block
grid_size = ((totalRaters + block_size[1] - 1) // block_size[1], 
             (totalMovies + block_size[0] - 1) // block_size[0])


print(f"Matrix A shape: {matrixAMovies.shape}")
print(f"Matrix B shape: {matrixBUsers.shape}")

# Calculate the shared memory size
shared_mem_size = ctypes.sizeof(Row) * min(totalRaters * totalMoviesUsingPerPerson, block_size[0] * block_size[1])
# Run the parallel CUDA kernel
cuda.Context.synchronize()  # Ensure synchronization before execution]
print("A GPU Memory Address:", int(A_gpu_matrixAMovies))
print("B GPU Memory Address:", int(B_gpu_matrixBUsers))
print("Rows GPU Memory Address:", int(rows_gpu))

# start_time = time.time()
# matmul(A_gpu_matrixAMovies, B_gpu_matrixBUsers, np.int32(N), np.int32(totalRaters), np.int32(totalMovies), rows_gpu,learning_rate,
#        block=block_size, grid=grid_size, shared=shared_mem_size)
# cuda.Context.synchronize()  # Ensure synchronization after execution
# end_time = time.time()
# # Copy result back to CPU

# # 🚀 First, copy updated A and B back to CPU
# matrixAMovies_result = np.empty_like(matrixAMovies)
# matrixBUsers_result = np.empty_like(matrixBUsers)

# cuda.memcpy_dtoh(matrixAMovies_result, A_gpu_matrixAMovies)
# cuda.memcpy_dtoh(matrixBUsers_result, B_gpu_matrixBUsers)
# rows_result = np.zeros(len(rows_data), dtype=[('col1', np.int32), ('col2', np.float32)])
# cuda.memcpy_dtoh(rows_result, rows_gpu)




200949
292758
Total rating used 24888158
Matrix A shape: (292758, 50)
Matrix B shape: (50, 200949)
A GPU Memory Address: 47343206400
B GPU Memory Address: 47401926656
Rows GPU Memory Address: 47443869696


In [2]:
dictMovieIDtoName={}
i=0
for row in matrixMovies:
    if row[0]==0:
        continue

    if int(row[0]) not in dictMovieIDtoName:
        dictMovieIDtoName[int(row[0])] = [row[1],row[2]]
    


In [3]:
epochs = 8  # Number of iterations

initial_lr = np.float32(0.001)  # Start slightly lower than 0.003
decay_factor = np.float32(0.95)   # Reduce by 10% every 5 epochs
decay_step = 5

min_lr = np.float32(0.0005)  # Ensure learning rate never goes too low
def normalize_embeddings(matrix):
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)  # Compute row-wise norm
    norms[norms == 0] = 1  # Prevent division by zero
    return matrix / norms
learning_rate = np.float32(0.001)
matrixAMovies_result=None
matrixBUsers_result=None
rows_result=None
for epoch in range(epochs):
    # learning_rate = np.float32(initial_lr * np.exp(-0.05 * epoch))
    if epoch==5:
        learning_rate = np.float32(0.0005)


    matrixAMovies_result = np.empty_like(matrixAMovies)
    matrixBUsers_result = np.empty_like(matrixBUsers)
    rows_result = np.zeros(len(rows_data), dtype=[('col1', np.int32), ('col2', np.float32)])
    start_time = time.time()
    # if epoch % 5 == 0:  # Every 5 epochs, reduce learning rate
    #     learning_rate *= 0.9  # Reduce by 10%

    matmul(A_gpu_matrixAMovies, B_gpu_matrixBUsers, np.int32(N), np.int32(totalRaters),
           np.int32(totalMovies), rows_gpu, learning_rate, block=block_size, grid=grid_size,
           shared=shared_mem_size)
    
    cuda.Context.synchronize()  # Wait for CUDA execution

    # Copy updated matrices and rows back to CPU
    cuda.memcpy_dtoh(matrixAMovies_result, A_gpu_matrixAMovies)
    cuda.memcpy_dtoh(matrixBUsers_result, B_gpu_matrixBUsers)
    cuda.memcpy_dtoh(rows_result, rows_gpu)


    if np.isnan(matrixAMovies_result).any() or np.isnan(matrixBUsers_result).any():
        print("Error: NaN detected in A or B matrix!")
        exit()

    if np.isinf(matrixAMovies_result).any() or np.isinf(matrixBUsers_result).any():
        print("Error: Inf detected in A or B matrix!")
        exit()

    # Compute loss
    mse = 0
    count = 0
    end_time = time.time()
    for i in range(len(rows_result)):
        if rows_result[i]['col1'] > 0:
            actual = rows_result[i]['col2']
            predicted = np.dot(matrixAMovies_result[rows_result[i]['col1'], :], matrixBUsers_result[:, i // totalMoviesUsingPerPerson])
            
            if np.isnan(predicted) or np.isinf(predicted):
                print(f"Error in prediction: row {i}, actual {actual}, predicted {predicted}")
                exit()

            mse += (actual - predicted) ** 2
            count += 1

    mse = mse / count if count > 0 else float('inf')  # Avoid division by zero

    
    print(f"Epoch {epoch+1}/{epochs}, MSE: {mse:.5f}, Time: {end_time - start_time:.5f} sec")





# Track execution time for CUDA

print(f"Parallel GPU Execution Time (CUDA): {end_time - start_time:.5f} seconds")


user_id = 2  # Example user (change as needed)
movie_id = rows_result[17]['col1']  # Get the movie ID from rows_result

predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}")







listUnused =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==0:
        listUnused.append([int(m[1]),int(m[2])])

listUnused2 =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==1:
        listUnused2.append([int(m[1]),int(m[2])])


for i in range(len(listUnused)):
    user_id = 2  # Example user (change as needed)
    movie_id = listUnused[i][0]  # Get the movie ID from rows_result
    actualPred = listUnused[i][1]
    predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
    print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}     Actual {actualPred}")

print()

Epoch 1/8, MSE: 4.86309, Time: 11.40502 sec
Epoch 2/8, MSE: 2.01153, Time: 11.18773 sec
Epoch 3/8, MSE: 1.61868, Time: 11.19290 sec
Epoch 4/8, MSE: 1.47795, Time: 11.14469 sec
Epoch 5/8, MSE: 1.40564, Time: 11.15890 sec
Epoch 6/8, MSE: 1.37620, Time: 11.11991 sec
Epoch 7/8, MSE: 1.35281, Time: 11.07519 sec
Epoch 8/8, MSE: 1.33853, Time: 11.12678 sec
Parallel GPU Execution Time (CUDA): 11.12678 seconds
Predicted rating for user 2 and movie 306: 5.12
Predicted rating for user 2 and movie 1199: 4.82     Actual 2
Predicted rating for user 2 and movie 1200: 4.72     Actual 5
Predicted rating for user 2 and movie 1203: 5.05     Actual 5
Predicted rating for user 2 and movie 1204: 5.13     Actual 1
Predicted rating for user 2 and movie 1208: 4.82     Actual 5
Predicted rating for user 2 and movie 1210: 4.64     Actual 2
Predicted rating for user 2 and movie 1211: 4.88     Actual 2
Predicted rating for user 2 and movie 1213: 4.83     Actual 5
Predicted rating for user 2 and movie 1217: 5.10   

In [4]:
print("HERE")
print()
for i in range(len(listUnused2)):
    user_id = 2  # Example user (change as needed)
    movie_id = listUnused2[i][0]  # Get the movie ID from rows_result
    actualPred = listUnused2[i][1]
    predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
    print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}     Actual {actualPred}")

HERE

Predicted rating for user 2 and movie 31: 4.23     Actual 5
Predicted rating for user 2 and movie 34: 4.84     Actual 5
Predicted rating for user 2 and movie 39: 4.30     Actual 5
Predicted rating for user 2 and movie 48: 3.76     Actual 5
Predicted rating for user 2 and movie 153: 3.76     Actual 3
Predicted rating for user 2 and movie 185: 4.13     Actual 5
Predicted rating for user 2 and movie 186: 3.88     Actual 5
Predicted rating for user 2 and movie 193: 2.90     Actual 3
Predicted rating for user 2 and movie 207: 4.34     Actual 5
Predicted rating for user 2 and movie 216: 4.05     Actual 4
Predicted rating for user 2 and movie 218: 4.18     Actual 5
Predicted rating for user 2 and movie 222: 4.43     Actual 5
Predicted rating for user 2 and movie 225: 4.34     Actual 5
Predicted rating for user 2 and movie 231: 3.35     Actual 2
Predicted rating for user 2 and movie 236: 4.39     Actual 4
Predicted rating for user 2 and movie 237: 4.21     Actual 5
Predicted rating for u

In [5]:
learning_rate = np.float32(0.001)

In [6]:
for epoch in range(2):

    cuda.memcpy_htod(rows_gpu, rows_result)

    cuda.memcpy_htod(A_gpu_matrixAMovies, matrixAMovies_result)
    cuda.memcpy_htod(B_gpu_matrixBUsers, matrixBUsers_result)
    # learning_rate = np.float32(initial_lr * np.exp(-0.05 * epoch))



    matrixAMovies_result = np.empty_like(matrixAMovies)
    matrixBUsers_result = np.empty_like(matrixBUsers)
    rows_result = np.zeros(len(rows_data), dtype=[('col1', np.int32), ('col2', np.float32)])
    start_time = time.time()
    # if epoch % 5 == 0:  # Every 5 epochs, reduce learning rate
    #     learning_rate *= 0.9  # Reduce by 10%

    matmul(A_gpu_matrixAMovies, B_gpu_matrixBUsers, np.int32(N), np.int32(totalRaters),
           np.int32(totalMovies), rows_gpu, learning_rate, block=block_size, grid=grid_size,
           shared=shared_mem_size)
    
    cuda.Context.synchronize()  # Wait for CUDA execution

    # Copy updated matrices and rows back to CPU
    cuda.memcpy_dtoh(matrixAMovies_result, A_gpu_matrixAMovies)
    cuda.memcpy_dtoh(matrixBUsers_result, B_gpu_matrixBUsers)
    cuda.memcpy_dtoh(rows_result, rows_gpu)


    if np.isnan(matrixAMovies_result).any() or np.isnan(matrixBUsers_result).any():
        print("Error: NaN detected in A or B matrix!")
        exit()

    if np.isinf(matrixAMovies_result).any() or np.isinf(matrixBUsers_result).any():
        print("Error: Inf detected in A or B matrix!")
        exit()

    # Compute loss
    mse = 0
    count = 0
    for i in range(len(rows_result)):
        if rows_result[i]['col1'] > 0:
            actual = rows_result[i]['col2']
            predicted = np.dot(matrixAMovies_result[rows_result[i]['col1'], :], matrixBUsers_result[:, i // totalMoviesUsingPerPerson])
            
            if np.isnan(predicted) or np.isinf(predicted):
                print(f"Error in prediction: row {i}, actual {actual}, predicted {predicted}")
                exit()

            mse += (actual - predicted) ** 2
            count += 1

    mse = mse / count if count > 0 else float('inf')  # Avoid division by zero

    end_time = time.time()
    print(f"Epoch {epoch+1}/{epochs}, MSE: {mse:.5f}, Time: {end_time - start_time:.5f} sec")

Epoch 1/8, MSE: 1.32553, Time: 127.05007 sec
Epoch 2/8, MSE: 1.32025, Time: 125.72030 sec


In [7]:
listUnused =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==0:
        listUnused.append([int(m[1]),int(m[2])])

listUnused2 =[]
for m in (matrixrating[55:180]):
    hit=0
    for i in range(1*totalMoviesUsingPerPerson,2*totalMoviesUsingPerPerson):
        if int(m[1]) ==int(rows_data[i].col1):
            
            hit=1
            break
    if hit==1:
        listUnused2.append([int(m[1]),float(m[2])])


print("HERE")
print()
for i in range(len(listUnused2)):
    user_id = 2  # Example user (change as needed)
    movie_id = listUnused2[i][0]  # Get the movie ID from rows_result
    actualPred = listUnused2[i][1]
    predicted = np.dot(matrixAMovies_result[movie_id, :], matrixBUsers_result[:, user_id])
    print(f"Predicted rating for user {user_id} and movie {movie_id}: {predicted:.2f}     Actual {actualPred}")

HERE

Predicted rating for user 2 and movie 31: 4.30     Actual 5.0
Predicted rating for user 2 and movie 34: 5.03     Actual 5.0
Predicted rating for user 2 and movie 39: 4.41     Actual 5.0
Predicted rating for user 2 and movie 48: 3.81     Actual 5.0
Predicted rating for user 2 and movie 153: 3.73     Actual 3.0
Predicted rating for user 2 and movie 185: 4.20     Actual 5.0
Predicted rating for user 2 and movie 186: 3.93     Actual 5.0
Predicted rating for user 2 and movie 193: 2.96     Actual 3.0
Predicted rating for user 2 and movie 207: 4.44     Actual 5.0
Predicted rating for user 2 and movie 216: 4.05     Actual 4.0
Predicted rating for user 2 and movie 218: 4.20     Actual 5.0
Predicted rating for user 2 and movie 222: 4.45     Actual 5.0
Predicted rating for user 2 and movie 225: 4.34     Actual 5.0
Predicted rating for user 2 and movie 231: 2.97     Actual 2.0
Predicted rating for user 2 and movie 236: 4.50     Actual 4.0
Predicted rating for user 2 and movie 237: 4.26     A

In [8]:
dictTest={}

for key, val in d.items():
    dictTest[key] = sorted(d[key][totalMoviesUsingPerPerson:], key=lambda x: x[0])

In [9]:
def findMovie(id,rating=None):
    if rating!=None:
        print(f"{id} Movie name is {matrixMovies[id]} Rated {rating}")
    else:
        print(f"{id} Movie name is {matrixMovies[id]}")

In [10]:
v=0
z=0
for key,val in dictTest.items():
    v+=len(val)
    if len(val)==0:
        z+=1
print(v)
print(z)

7112046
183389


In [11]:
v=0
for key,val in newD.items():
    v+=len(val)
    
print(v)


24888158


In [12]:
24704769+7094487

31799256

In [13]:
print(len(matrixrating))

32000204


In [14]:
# mse=0
# count=0
# for key,val in dictTest.items():

#     userID=key
#     for row in val:
#         movieID=int(row[0])
#         actual = float(row[1])
#         predicted = np.dot(matrixAMovies_result[movieID, :], matrixBUsers_result[:, userID // totalMoviesUsingPerPerson])
            
#         if np.isnan(predicted) or np.isinf(predicted):
#             print(f"Error in prediction: row {i}, actual {actual}, predicted {predicted}")
#             exit()

#         mse += (actual - predicted) ** 2
#         count += 1
#         print(f"Predicted rating for user {userID} and movie {movieID}: {predicted:.2f}     Actual {actual}")
        
# mse = mse / count if count > 0 else float('inf')  # Avoid division by zero

# print(f"MSE: {mse:.5f}")

KNN

In [15]:
def normalize_embeddings(matrix):
    norms = np.linalg.norm(matrix, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Avoid division by zero
    return matrix / norms

import numpy as np
import faiss

normalized_movies=matrixAMovies_result
# Build FAISS index
d = normalized_movies.shape[1]
index = faiss.IndexFlatIP(d)
index.add(normalized_movies)

# def compute_cosine_similarity(matrix):
#     return np.dot(matrix, matrix.T)

# similarity_matrix = compute_cosine_similarity(normalized_movies)


In [16]:
m=1371
#40815
#260
#1371
#1994

# Movie ID 1 is at row index 1 (if 0-based index)
query_vector = normalized_movies[m].reshape(1, -1)  # FAISS expects (batch_size, dim)

k = 10  # Number of similar movies you want
D, I = index.search(query_vector, k)

print("Top", k, "similar movies to movie:")
findMovie(m)
print()
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    # print(f"{rank + 1}: Movie ID {idx} with similarity {similarity:.4f}")
    findMovie(idx)


Top 10 similar movies to movie:
1371 Movie name is [1371, 'Star Trek: The Motion Picture (1979)', 'Adventure|Sci-Fi']

597 Movie name is [597, 'Pretty Woman (1990)', 'Comedy|Romance']
1101 Movie name is [1101, 'Top Gun (1986)', 'Action|Romance']
1721 Movie name is [1721, 'Titanic (1997)', 'Drama|Romance']
356 Movie name is [356, 'Forrest Gump (1994)', 'Comedy|Drama|Romance|War']
733 Movie name is [733, '"Rock', 'Action|Adventure|Thriller']
780 Movie name is [780, 'Independence Day (a.k.a. ID4) (1996)', 'Action|Adventure|Sci-Fi|Thriller']
2000 Movie name is [2000, 'Lethal Weapon (1987)', 'Action|Comedy|Crime|Drama']
1356 Movie name is [1356, 'Star Trek: First Contact (1996)', 'Action|Adventure|Sci-Fi|Thriller']
1393 Movie name is [1393, 'Jerry Maguire (1996)', 'Drama|Romance']
1291 Movie name is [1291, 'Indiana Jones and the Last Crusade (1989)', 'Action|Adventure']


In [17]:
def get_top_k_similar_movies_single(movie_id, normalized_movies, k=5):
    movie_vector = normalized_movies[movie_id]
    similarities = np.dot(normalized_movies, movie_vector)  # Only one movie
    top_k_indices = np.argsort(similarities)[-k-1:-1][::-1]  # Exclude itself
    return top_k_indices

# Usage:
top_k = get_top_k_similar_movies_single(m, normalized_movies=normalized_movies, k=5)
print(top_k)
    

[1101 1721  356  733  780]


In [18]:


findMovie(123)

123 Movie name is [123, 'Chungking Express (Chung Hing sam lam) (1994)', 'Drama|Mystery|Romance']


In [19]:
for id in top_k:
    findMovie(id)

1101 Movie name is [1101, 'Top Gun (1986)', 'Action|Romance']
1721 Movie name is [1721, 'Titanic (1997)', 'Drama|Romance']
356 Movie name is [356, 'Forrest Gump (1994)', 'Comedy|Drama|Romance|War']
733 Movie name is [733, '"Rock', 'Action|Adventure|Thriller']
780 Movie name is [780, 'Independence Day (a.k.a. ID4) (1996)', 'Action|Adventure|Sci-Fi|Thriller']


In [20]:

# Search for similar movies
movie_id = 72
k = 5
query_vector = normalized_movies[movie_id:movie_id+1]

distances, indices = index.search(query_vector, k)


findMovie(movie_id)
print(f"Top {k} similar movies to movie {movie_id} are: {indices.flatten()}")
print()

for id in indices.flatten():
    print(id,end=" ")
    findMovie(id)

    

72 Movie name is [72, 'Kicking and Screaming (1995)', 'Comedy|Drama']
Top 5 similar movies to movie 72 are: [ 318   50  296 2019 1203]

318 318 Movie name is [318, '"Shawshank Redemption', 'Crime|Drama']
50 50 Movie name is [50, '"Usual Suspects', 'Crime|Mystery|Thriller']
296 296 Movie name is [296, 'Pulp Fiction (1994)', 'Comedy|Crime|Drama|Thriller']
2019 2019 Movie name is [2019, 'Seven Samurai (Shichinin no samurai) (1954)', 'Action|Adventure|Drama']
1203 1203 Movie name is [1203, '12 Angry Men (1957)', 'Drama']


In [21]:
d2 = matrixBUsers_result.shape[1]
index = faiss.IndexFlatIP(d2)
index.add(matrixBUsers_result)

In [22]:
# Movie ID 1 is at row index 1 (if 0-based index)
userID=1

query_vector = matrixBUsers_result[userID].reshape(1, -1)  # FAISS expects (batch_size, dim)

k = 10  # Number of similar movies you want
D, I = index.search(query_vector, k)

print("Top", k, "similar movies to movie ID",userID,":")
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    print(f"{rank + 1}: User ID {idx} with similarity {similarity:.4f}")


Top 10 similar movies to movie ID 1 :
1: User ID 45 with similarity 6939.9365
2: User ID 38 with similarity 6504.5747
3: User ID 9 with similarity 5979.7373
4: User ID 46 with similarity 5886.7642
5: User ID 32 with similarity 5828.4419
6: User ID 24 with similarity 5669.2910
7: User ID 8 with similarity 5180.0063
8: User ID 1 with similarity 4865.7280
9: User ID 23 with similarity 4799.7046
10: User ID 42 with similarity 4325.0762


In [23]:
user_embeddings = matrixBUsers_result.T  # Now shape (totalRaters, N)

# Build the FAISS index
d = user_embeddings.shape[1]  # Dimensionality (N)
index = faiss.IndexFlatIP(d)  # Inner product (good for cosine similarity)
index.add(user_embeddings)  # Add user vectors

# --- QUERY PART ---
userID = 1  # User ID you want neighbors for

query_vector = user_embeddings[userID].reshape(1, -1).astype(np.float32)  # FAISS expects (batch_size, dim)

k = 10  # Number of similar users you want
D, I = index.search(query_vector, k)

print(f"Top {k} similar users to user ID {userID}:")
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    print(f"{rank + 1}: User ID {idx} with similarity {similarity:.4f}")

Top 10 similar users to user ID 1:
1: User ID 41450 with similarity 2.6454
2: User ID 80472 with similarity 2.3766
3: User ID 87007 with similarity 2.2391
4: User ID 49805 with similarity 2.1042
5: User ID 144129 with similarity 1.9628
6: User ID 145780 with similarity 1.8190
7: User ID 152132 with similarity 1.8166
8: User ID 132423 with similarity 1.8141
9: User ID 165840 with similarity 1.7922
10: User ID 60356 with similarity 1.7850


In [24]:
for i in range(100):
    if rows_data[i]==None:
        break
    if rows_data[i].col2<4.4:
        continue
    print(dictMovieIDtoName[rows_data[i].col1],rows_data[i].col2)
    

['Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 'Crime|Drama'] 5.0
['Twelve Monkeys (a.k.a. 12 Monkeys) (1995)', 'Mystery|Sci-Fi|Thriller'] 5.0
['"White Balloon', 'Children|Drama'] 5.0
['Taxi Driver (1976)', 'Crime|Drama|Thriller'] 5.0
['"Doom Generation', 'Comedy|Crime|Drama'] 5.0
['Eat Drink Man Woman (Yin shi nan nu) (1994)', 'Comedy|Drama|Romance'] 5.0
['Star Wars: Episode IV - A New Hope (1977)', 'Action|Adventure|Sci-Fi'] 5.0
['Three Colors: Red (Trois couleurs: Rouge) (1994)', 'Drama'] 5.0
['Three Colors: Blue (Trois couleurs: Bleu) (1993)', 'Drama'] 5.0
['Blade Runner (1982)', 'Action|Sci-Fi|Thriller'] 5.0
['Welcome to the Dollhouse (1995)', 'Comedy|Drama'] 5.0
['Emma (1996)', 'Comedy|Drama|Romance'] 5.0
['North by Northwest (1959)', 'Action|Adventure|Mystery|Romance|Thriller'] 5.0
['"Apartment', 'Comedy|Drama|Romance'] 5.0
['Sabrina (1954)', 'Comedy|Romance'] 5.0
['Citizen Kane (1941)', 'Drama|Mystery'] 5.0
['All About Eve (1950)', 'Drama'] 5.0
['"Women', 'Comedy'] 5.

In [25]:
for i in range(len(I[0])//2):
    idx=I[0][i]
    print(f"USER ID {idx}")
    for j in range(100):
            if rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col1==0:
                  break
            if (rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col2<4.4):
                  continue
            findMovie(rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col1,rows_data[(idx-1)*totalMoviesUsingPerPerson+j].col2)
            

USER ID 41450
4447 Movie name is [4447, 'Legally Blonde (2001)', 'Comedy|Romance'] Rated 5.0
6535 Movie name is [6535, '"Legally Blonde 2: Red', 'Comedy'] Rated 5.0
8453 Movie name is [8453, 'It Had to Be You (2000)', 'Comedy|Romance'] Rated 5.0
8643 Movie name is [8643, '"Cinderella Story', 'Comedy|Romance'] Rated 5.0
55245 Movie name is [55245, 'Good Luck Chuck (2007)', 'Comedy|Romance'] Rated 5.0
107563 Movie name is [107563, '"Princess for Christmas', 'Children|Comedy'] Rated 5.0
137162 Movie name is [137162, 'Beside Still Waters (2013)', '(no genres listed)'] Rated 5.0
148857 Movie name is [148857, '"Christmas', 'Drama|Romance'] Rated 5.0
149392 Movie name is [149392, '12 Gifts of Christmas (2015)', 'Children'] Rated 5.0
158900 Movie name is [158900, 'Autumn Dreams (2015)', 'Romance'] Rated 5.0
158940 Movie name is [158940, 'A Bride for Christmas (2012)', 'Comedy'] Rated 5.0
159435 Movie name is [159435, 'I Married Who? (2012)', 'Comedy'] Rated 5.0
162380 Movie name is [162380, "L

In [26]:
for i in range(totalMoviesUsingPerPerson):
    print(rows_data[(45-1)*totalMoviesUsingPerPerson+i].col1)

47
318
593
858
1136
1196
1200
1201
1214
1221
1232
1704
2019
2023
2571
3578
4993
5618
5952
7153
58559
79132
106489
122912
122916
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


This is to show that the gradients from the matrix Multiplication were able to find similar patterns in Movies. With random gradients it cannot find any patterns. With our updated gradients it finds similar movies.

In [27]:
###########################################TEST####################


matrixAMoviesTEST = np.random.uniform(-0.1, 0.1, size=(totalMovies, N)).astype(np.float32)
matrixBUsersTEST = np.random.uniform(-0.1, 0.1, size=(N, totalRaters)).astype(np.float32)

In [28]:
# Assume matrixAMovies_result is (totalMovies, N)

# 1. Build the FAISS index
d = matrixAMoviesTEST.shape[1]  # Dimensionality (N)
index = faiss.IndexFlatIP(d)  # IP = Inner Product (cosine similarity if vectors normalized)
index.add(matrixAMoviesTEST)  # Add all movie embeddings to index

# 2. Query the FAISS index
movie_id = 40815  # Some movie ID you want neighbors for
query = matrixAMoviesTEST[movie_id].reshape(1, -1).astype(np.float32)

k = 5  # Number of neighbors
D, I = index.search(query, k)

print("Top K similar movie IDs:", I)
print("Similarities:", D)
print("Top", k, "similar movies to movie:")
findMovie(40815)
print()
for rank, (idx, similarity) in enumerate(zip(I[0], D[0])):
    # print(f"{rank + 1}: Movie ID {idx} with similarity {similarity:.4f}")
    findMovie(idx)

Top K similar movie IDs: [[ 40815  96147 143799 124498 230457]]
Similarities: [[0.17318572 0.10343668 0.10027855 0.09912314 0.09646774]]
Top 5 similar movies to movie:
40815 Movie name is [40815, 'Harry Potter and the Goblet of Fire (2005)', 'Adventure|Fantasy|Thriller|IMAX']

40815 Movie name is [40815, 'Harry Potter and the Goblet of Fire (2005)', 'Adventure|Fantasy|Thriller|IMAX']
96147 Movie name is [0, '', '']
143799 Movie name is [143799, "The Priest's Wife (1970)", 'Comedy']
124498 Movie name is [124498, 'They Call It Sin (1932)', 'Drama']
230457 Movie name is [0, '', '']
